<a href="https://colab.research.google.com/github/AKookani/Video-Retrieval/blob/main/Video_Retrival.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Install Required Libraries

In [1]:
!pip install datasets
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.7 MB/s eta 0:00:00


In [2]:
!pip install sentence-transformers

In [3]:
!pip install datasets pytube opencv-python-headless transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.3 MB/s eta 0:00:00


In [4]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 44.6 MB/s eta 0:00:00


## 2. Download the dataset and check it out

In [5]:
from datasets import load_dataset

ds = load_dataset("lmms-lab/YouCook2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/608 [00:00<?, ?B/s]

val-00000-of-00001.parquet:   0%|          | 0.00/168k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/43.7k [00:00<?, ?B/s]

Generating val split:   0%|          | 0/3179 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1471 [00:00<?, ? examples/s]

In [6]:
ds

DatasetDict({
    val: Dataset({
        features: ['id', 'video_url', 'recipe_type', 'segment', 'sentence', 'video_path', 'youtube_id'],
        num_rows: 3179
    })
    test: Dataset({
        features: ['id', 'video_url', 'recipe_type', 'segment', 'sentence', 'video_path', 'youtube_id'],
        num_rows: 1471
    })
})

In [7]:
len(ds['val']['sentence']), len(ds['test']['sentence'])

(3179, 1471)

## 3. Translate the annotations

In [ ]:
from deep_translator import GoogleTranslator

# Group sentences into smaller batches
batch_size = 50
sentences = ds['val']['sentence']
batches = [sentences[i:i + batch_size] for i in range(0, len(sentences), batch_size)]

translated_sentences = []
translator = GoogleTranslator(source='en', target='fa')

# Translate each batch
for batch in batches:
    joined_sentences = "\n".join(batch)  # Join sentences with a delimiter (e.g., newline)
    translated_batch = translator.translate(joined_sentences)  # Translate the batch
    translated_sentences.extend(translated_batch.split("\n"))  # Split back into individual sentences

# Result
translated_sentence = translated_sentences
translated_sentence

['انتهای وردالاگو را انتخاب کنید',
 'آب لیمو نمک سیر سماق و روغن را در یک کاسه مخلوط کنید',
 'کاهو را خرد کرده و در ظرفی بریزید',
 'وردالاگو فلفل خیار گوجه فرنگی سبزی و پیاز را به کاهو داخل کاسه اضافه کنید',
 'سس را روی سالاد بریزید و مخلوط کنید',
 'پیتای سرخ شده را به سالاد اضافه کنید و مخلوط کنید',
 'شیر کره نمک و سس تند را در یک کاسه مخلوط کنید',
 'مرغ را در یک کاسه کوچک قرار دهید و روی مرغ آب نمک بریزید و آن را به مدت 1 ساعت در فریزر قرار دهید',
 'آب نمک را بریزید و مرغ را کاملاً بشویید و آب اضافی را تکان دهید',
 'مرغ را در بشقاب یا سینی بچینید و سخاوتمندانه با ادویه های مخلوط مزه دار کنید',
 'آرد را با نمک فلفل و نمک کرفس مزه دار کنید',
 'تکه های مرغ را در آرد بپوشانید',
 'مرغ را در روغن داغ بریزید و سرخ کنید تا طلایی شود',
 'گوشت های تیره را با هم و گوشت های سفید را با هم تفت دهید',
 'پودر نان و پنیر پارمزان را مخلوط کنید',
 'مرغ را بکوبید',
 'نمک و فلفل را روی مرغ بمالید',
 'آرد را روی مرغ بمالید و آن را در تخم مرغ آغشته کنید و با آرد سوخاری بپوشانید',
 'مرغ را در تابه سرخ کنید'

In [ ]:
ds['val']['sentence'][:]

['pick the ends off the verdalago',
 'combine lemon juice sumac garlic salt and oil in a bowl',
 'chop lettuce and place it in a bowl',
 'add verdalago pepper cucumbers tomatoes herbs and onions to the lettuce in the bowl',
 'pour the dressing over the salad and mix',
 'add the fried pita to the salad and mix',
 'combine salt butter milk and hot sauce in a bowl',
 'place chicken in a small bowl and pour brine over chicken and freeze it for 1 hour',
 'pour off the brine and rinse chicken off thoroughly and shake off excess water',
 'place chicken on a plate or tray and season generously with mixed spices',
 'season the flour with salt pepper and celery salt',
 'coat the chicken pieces in the flour',
 'place chicken in hot oil and fry until golden brown',
 'fry dark meats together and white meats together',
 'mix bread crumbs and parmesan cheese',
 'pound the chicken',
 'rub salt and pepper onto the chicken',
 'rub flour onto the chicken  dip it in egg and coat with breadcrumbs',
 'fry t

## 4. Prepare the text embeddings and check their precsion out

In [8]:
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cdist
import statistics

# Load the model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

# Example lists (replace with your actual lists)
english_sentences = ds['val']['sentence']
translated_sentence = translated_sentence  # Your translated sentences in Persian

# Encode all English and Persian sentences in batches
english_embeddings = model.encode(english_sentences, batch_size=64)
persian_embeddings = model.encode(translated_sentence, batch_size=64)

# Compute cosine similarity in bulk using scipy's cdist
# 1 - cosine similarity since cdist calculates cosine distance (1 - similarity)
similarity_scores = 1 - cdist(english_embeddings, persian_embeddings, metric='cosine').diagonal()
avg = statistics.mean(similarity_scores)
# Result: similarity_scores contains the similarity for each sentence pair
similarity_scores, avg

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(array([0.83166427, 0.82929314, 0.79953221, ..., 0.81652877, 0.9581216 ,
        0.94662812]),
 0.8225685641445796)

## 5. Prepare the video embeddings

### 5.1 Download Videos for Segments

In [10]:
import os
import subprocess

def download_video(youtube_id, save_path="videos/"):
    os.makedirs(save_path, exist_ok=True)
    url = f"https://www.youtube.com/watch?v={youtube_id}"
    output_template = os.path.join(save_path, f"{youtube_id}.mp4")
    try:
        subprocess.run(["yt-dlp", "-f", "best[ext=mp4]", "-o", output_template, url], check=True)
        return output_template
    except subprocess.CalledProcessError as e:
        print(f"Error downloading {youtube_id}: {e}")
        return None

In [11]:
import requests

def is_video_available(youtube_id):
    url = f"https://www.youtube.com/watch?v={youtube_id}"
    response = requests.get(url)
    return response.status_code == 200

### 5.2 Extract Frames for Segments

In [12]:
import cv2
import os

def extract_frames(video_path, start, end, frame_rate=1, output_folder="frames/"):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        time = frame_count / fps
        if start <= time <= end:
            if frame_count % (fps // frame_rate) == 0:  # Adjust to save every nth frame
                frame_name = os.path.join(output_folder, f"frame_{frame_count}.jpg")
                cv2.imwrite(frame_name, frame)

        frame_count += 1
        if time > end:
            break

    cap.release()

### 5.3 Process and Extract Embeddings

In [13]:
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image
import glob

# Load CLIP
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def get_frame_embeddings(frame_folder):
    frame_paths = glob.glob(os.path.join(frame_folder, "*.jpg"))
    frame_paths.sort()  # Ensure frame order is consistent
    embeddings = []

    for frame_path in frame_paths:
        image = Image.open(frame_path).convert("RGB")
        inputs = processor(images=image, return_tensors="pt")
        outputs = model.get_image_features(**inputs)
        embeddings.append(outputs)

    return torch.stack(embeddings)  # Shape: (num_frames, embedding_dim)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

### 5.4 Pipeline Integration
Combine all steps:
1. Download the video using `download_video`.
2. Extract frames using `extract_frames`.
3. Compute embeddings using `get_frame_embeddings`.

In [14]:
segments = ds["val"]["segment"]
sentences = ds["val"]["sentence"]
video_ids = ds["val"]["youtube_id"]

metadata = [
    {"youtube_id": video_id, "start": seg[0], "end": seg[1], "sentence": sent}
    for video_id, seg, sent in zip(video_ids, segments, sentences)
]

In [15]:
for meta in metadata[:5]:  # Process the first 5 videos
    video_file = download_video(meta["youtube_id"])
    extract_frames(video_file, meta["start"], meta["end"], frame_rate=1)
    embeddings = get_frame_embeddings("frames/")
    print(f"Processed video {meta['youtube_id']} with sentence: {meta['sentence']}")

Processed video xHr8X2Wpmno with sentence: pick the ends off the verdalago
Processed video xHr8X2Wpmno with sentence: combine lemon juice sumac garlic salt and oil in a bowl
Processed video xHr8X2Wpmno with sentence: chop lettuce and place it in a bowl
Processed video xHr8X2Wpmno with sentence: add verdalago pepper cucumbers tomatoes herbs and onions to the lettuce in the bowl
Processed video xHr8X2Wpmno with sentence: pour the dressing over the salad and mix


In [16]:
embeddings

tensor([[[-0.5357, -0.1951,  0.4790,  ...,  0.7901,  0.0387,  0.2240]],

        [[-0.2968, -0.4536,  0.4555,  ...,  0.7109, -0.0050,  0.2773]],

        [[-0.3974, -0.5114,  0.5380,  ...,  0.4166,  0.0230,  0.3281]],

        ...,

        [[-0.6861,  0.1142,  0.2495,  ...,  0.3653, -0.0991,  0.2237]],

        [[-0.7386,  0.1093,  0.2085,  ...,  0.4666, -0.1967,  0.2071]],

        [[-0.5015,  0.0232,  0.3210,  ...,  0.6496, -0.2158,  0.1140]]],
       grad_fn=<StackBackward0>)

In [9]:
import pandas as pd

# Create a dictionary with your data
data = {
    'English_Text': ds['val']['sentence'],      # Original English sentences
    'Persian_Text': translated_sentence,       # Translated Persian sentences
    'Similarity_Score': similarity_scores      # Similarity scores
}

# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file on the disk
output_file = '/content/translation_quality.csv'  # Saves to Colab's default file system
df.to_csv(output_file, index=False, encoding='utf-8')

print(f"File saved at: {output_file}")

File saved at: /content/translation_quality.csv
